# Feature Engineering Code-Along

<a href="https://colab.research.google.com/github/coding-dojo-data-science/week-10-lecture-2-feature-engineering/blob/11-7-22/Feature%20Engineering%20Code-along.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook we use the following feature engineering strategies:
1. scaling
2. binning
3. removing outliers
4. log transformation
5. target encoding
6. polynomial features

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error, \
precision_score, recall_score, accuracy_score, f1_score, ConfusionMatrixDisplay, \
classification_report

import warnings
warnings.filterwarnings('ignore')

# Useful Functions

In [ ]:
def eval_regression(true, pred, name='Model'):
    """Evaluates true and predicted values from a regression model.  
    Outputs a dataframe of metrics"""
    scores = pd.DataFrame()
    scores['Model Name'] = [name]
    scores['RMSE'] = [np.sqrt(mean_squared_error(true, pred))]
    scores['MAE'] = [mean_absolute_error(true, pred)]
    scores['R2'] = [r2_score(true, pred)]
    scores.set_index('Model Name', inplace=True)

    return scores

def eval_classification(true, pred, name='Model'):
    """shows classification_report and confusion matrix
    for classification model predictions.  Outputs a dataframe of metrics"""
  
    print(name, '\n')
    print(classification_report(true, pred))
    ConfusionMatrixDisplay.from_predictions(true, pred)
    plt.show()

    scores = pd.DataFrame()
    scores['Model Name'] = [name]
    scores['Precision'] = [precision_score(true, pred)]
    scores['Recall'] = [recall_score(true, pred)]
    scores['F1 Score'] = [f1_score(true, pred)]
    scores['Accuracy'] = [accuracy_score(true, pred)]
    scores.set_index('Model Name', inplace=True)

    return scores

## Data

Today we will use data about used car sales in India from Kaggle.  [Here is the source](https://www.kaggle.com/datasets/saisaathvik/used-cars-dataset-from-cardekhocom)

In [ ]:
# load data
df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vTeFV8-y5rNxCM946otPPyTF5UZ5rIOTOHrXCYU5zkqB7DubjxGKGzaxVayeLdU0wcCqLMrhjUbgkEp/pub?gid=1641648108&single=true&output=csv')
df_backup = df.copy()

display(df.head())
print(df.shape)

## Explore and clean the data

In [ ]:
# drop 'Id' column
df.drop(columns='Id', inplace=True)
df.head()

In [ ]:
# check for duplicates
df.duplicated().sum()

In [ ]:
# remove duplicates
df.drop_duplicates(inplace=True)
df.duplicated().sum()

In [ ]:
# check for missing values
df.isna().sum()

In [ ]:
# check ALL summary statistics


## Some Numeric Feature Engineering

In [ ]:
# explore numeric distributions


### Filter to remove outliers

In [ ]:
# check year value_counts

In [ ]:
# exclude cars made before 2000


## Convert year to nominal categorical

This way, if cars made in certain years are more expensive than those made in other years, the model can see this in a non-linear way.

In [ ]:
# convert year to categorical


## Transform to normalize feature

In [ ]:
# Check the histogram for 'distance_travelled(kms)'


In [ ]:
# log transform 'distance_travelled(kms)'


## Binning the Target: Let's change this regression problem to a binary classification problem

In [ ]:
# explore statistics of price with .describe()


#### What's wrong with the code below?  How would you fix it?

In [ ]:
# encode cars with prices above the mean as 1 and below the mean as 0



In [ ]:
X = df.drop(columns=['price', 'is_expensive'])
y = df['price']

X_train, X_test, y_train, y_test =  train_test_split(X, y, random_state=42)

# Bin the Target

In [ ]:
df['price'].head()

In [ ]:
# Bin price into cars below the mean price and those above the mean price


In [ ]:
# Check the value counts of the new labels


# Base Model

In [ ]:
# instantiate transformers
scaler = StandardScaler()
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')

cat_cols = make_column_selector(dtype_include='object')
num_cols = make_column_selector(dtype_include='number')

# create preprocessing pipeline
base_preprocess = make_column_transformer((ohe, cat_cols), (scaler, num_cols))

# Examine the results of all transformers
base_preprocess.fit_transform(X_train, y_train).shape

In [ ]:
# Create and evaluate model with a base logistic regression
logreg_base = LogisticRegression(max_iter=1000)
logreg_base = make_pipeline(base_preprocess, logreg_base)
logreg_base.fit(X_train, y_train)

train_pred = logreg_base.predict(X_train)
test_pred = logreg_base.predict(X_test)

In [ ]:
# evaluate model with eval_classification


# Modeling: PolynomialFeatures

## Test PolynomialFeatures to see how it changes data

We are working here with the entire dataframe to illustrate how the model combines numeric features.  We won't apply the transformers until later.

In [ ]:
# Scale the data


# Create polynomial features


In [ ]:
# Plot distance vs distance^2
plt.scatter(data=poly_train, x='distance_travelled(kms)', y='distance_travelled(kms)^2')

In [ ]:
# Plot distance vs distance^2
plt.scatter(data=poly_train, x='distance_travelled(kms)', y='distance_travelled(kms)^3')

## Model with PolynomialFeatures

In [ ]:
# instantiate transformers
scaler = StandardScaler()
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
# Instantiate the PolynomialFeatures transformer
poly = PolynomialFeatures(degree=2)

cat_cols = make_column_selector(dtype_include='object')
num_cols = make_column_selector(dtype_include='number')

# Add the PolynomialFeatures transformer to the numeric pipeline
num_pipe = make_pipeline(scaler, poly)

# create preprocessing pipeline
preprocess = make_column_transformer((ohe, cat_cols), (num_pipe, num_cols))

# Examine the results of all transformers
preprocess.fit_transform(X_train, y_train).shape

In [ ]:
# Create and evaluate model with PolynomialFeatures
poly_logreg = LogisticRegression(max_iter=1000)
poly_logreg = make_pipeline(preprocess, poly_logreg)
poly_logreg.fit(X_train, y_train)

train_pred = poly_logreg.predict(X_train)
test_pred = poly_logreg.predict(X_test)

In [ ]:
# evaluate model


## Compare base and polynomial model scores

## What if we change the degree?